In [78]:
# Purpose:  Model topics for group 3 ( 5th democratic debate and 8th republican )
# Author:  Preeti Javaji
# Date:  05/08/16
# Course:  MAS DSE capstone, Spring 2016

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import math

pd.set_option('display.max_colwidth', 240)
pd.set_option('display.max_columns', 50)

import sys
import os
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB


print 'done'

done


In [80]:
# Function defs.

# return string with non-ascii chars removed
def to_ascii(s):
    returnstr = s.strip()
    returnstr = "".join([ch for ch in returnstr if ord(ch)< 128])
    return returnstr

print 'done'

done


In [81]:
# Load data file & clean the 'text' column.

df = pd.read_csv("Sentiment_backup.csv")

# Convert to ascii
df['ascii'] = df['text'].apply(to_ascii)
#df['ascii'] = df['ascii'].str.encode('ascii',errors='ignore') 
df['ascii'] = df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
df['ascii'] = df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
df['ascii'] = df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
df['ascii'] = df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
df['ascii'] = df['ascii'].str.lower()

df['K_sentiment'] = df['sentiment']
print df.shape
#print df[df['ascii']!=df['text']][['ascii', 'K_sentiment']].head(10)
df[['text', 'ascii']].head(3)
#df[df['ascii'].str.contains("\n")][['text', 'ascii']].head(10)

(4333, 23)


,text,ascii
0,RT @TheBaxterBean: Scott Walker's Abortion Ban Allows Rapist Father To Sue For Emotional Distress http://t.co/rHMvgumuir #GOPDebate @Badger��_,rt @thebaxterbean scott walkers abortion ban allows rapist father to sue for emotional distress url #gopdebate @badger_
1,"RT @mydaughtersarmy: Dear GOP,\rIf you want to stop abortions... \r\r#GOPDebate \rhttp://t.co/RbL1t4iPx6","rt @mydaughtersarmy dear gop,\rif you want to stop abortions... \r\r#gopdebate \rurl"
2,RT @Bipartisanism: A message to the #GOPDebate candidates: http://t.co/WFK5CmVgiN,rt @bipartisanism a message to the #gopdebate candidates url


In [82]:
# Show the class frequencies.
counts = df['subject_matter'].value_counts()
total = counts.sum()

counts = counts.apply(lambda x: ("Count: %d" % x, "Frequency: %.2f" % (float(x)/total)) )

#print("%.2f" % a)
#string = 'string%d' % (i,)

print counts
print type(counts)

None of the above                       (Count: 1271, Frequency: 0.33)
Religion                                 (Count: 407, Frequency: 0.11)
Women's Issues (not abortion though)     (Count: 362, Frequency: 0.10)
Racial issues                            (Count: 353, Frequency: 0.09)
Abortion                                 (Count: 293, Frequency: 0.08)
Jobs and Economy                         (Count: 251, Frequency: 0.07)
Immigration                              (Count: 211, Frequency: 0.06)
FOX News or Moderators                   (Count: 188, Frequency: 0.05)
LGBT issues                              (Count: 126, Frequency: 0.03)
iran deal                                 (Count: 74, Frequency: 0.02)
Healthcare (including Medicare)           (Count: 67, Frequency: 0.02)
Gun Control                               (Count: 61, Frequency: 0.02)
Foreign Policy                            (Count: 41, Frequency: 0.01)
ISIS                                      (Count: 40, Frequency: 0.01)
none  

In [51]:
%pwd

u'/Users/preetijavaji/Documents/DSE/capstone'

In [101]:
# Add the 1st democratic debate transcript
D1_df = pd.read_csv("labeled_transcripts/D5_proc_labeled.csv")
D1_df.head()

,Unnamed: 0,duration,speaker,speaker_type,start_time,text,topic
0,0,74,TODD,moderator,0,"TODD: Good evening, and welcome to the MSNBC Democratic candidates debate.",intro
1,1,236,MADDOW,moderator,74,"MADDOW: We are super excited to be here at the University of New Hampshire. Tonight, this is the first time that Hillary Clinton and Bernie Sanders have squared off exactly like this. Face-to-face, just one-on-one, just the two of them.",intro
2,2,354,TODD,moderator,310,"TODD: And neither party has seen this yet. These candidates are both running for the Democratic nomination, but they are very different from each other when it comes to what matters most and how they would go about the job of being pres...",intro
3,3,50,APPLAUSE,audience,1059,[applause],intro
4,4,395,MADDOW,moderator,664,MADDOW: And we do hope that the candidates will take this opportunity to show us the distinctions���show us the differences between them. That's the whole reason that we're here tonight. We're not here for talking points. We're to learn...,intro


In [102]:
#clean it up too
D1_df['ascii'] = D1_df['text'].apply(to_ascii)

D1_df['ascii'] = D1_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D1_df['ascii'] = D1_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D1_df['ascii'] = D1_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D1_df['ascii'] = D1_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D1_df['ascii'] = D1_df['ascii'].str.lower()

In [103]:
# Now read hashtag tweets
D2_df = pd.read_csv("labeled_transcripts/hashtagTweets_updated.csv")
D2_df.tail()

,topic,tweetid,text
1549,deathpenalty,NaN,""".@BernieSanders proud to stand w/ u FOR #unions #fairtrade &amp; Against #deathpenalty #TPP &amp; PACs/SuperPACs &amp; Establishment pols #nhpolitics """
1550,deathpenalty,NaN,"""@ckgeorgia @People4Bernie @BernieSanders Should also say, Bernie Sides with the #Pope on #Incarceration &amp; #DeathPenalty Issues! #FeelTheBern """
1551,deathpenalty,NaN,"""Liberals all oppose the #deathpenalty while thinking nothing of the murder of #unborn #alllivesmatter https://t.co/cEEGvc5Pt2 """
1552,deathpenalty,NaN,""".@BernieSanders &amp; I are against the death penalty as an ethical imperative. Let's get civilized in the US and end it forever #deathpenalty """
1553,deathpenalty,NaN,""".@BernieSanders thank you for standing against #DeathPenalty, please stand with us against drone strikes as well. https://t.co/7PIJ0rnLGO """


In [104]:
#clean it up
#clean it up too
D2_df['ascii'] = D2_df['text'].apply(to_ascii)

D2_df['ascii'] = D2_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D2_df['ascii'] = D2_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D2_df['ascii'] = D2_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D2_df['ascii'] = D2_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D2_df['ascii'] = D2_df['ascii'].str.lower()

In [105]:
# Read the next debate
D3_df = pd.read_csv("labeled_transcripts/R8_proc_labelled.csv")
D3_df.head()

,Unnamed: 0,duration,speaker,speaker_type,start_time,text,topic
0,0,372,MUIR,moderator,0,"MUIR: Good evening, again, everyone. This is the first time since Iowa and the only time before the New Hampshire primary that the Republican candidates will have the opportunity to face each other. The people of Iowa have been heard. N...",intro
1,1,262,RADDATZ,moderator,372,RADDATZ: Questions of leadership and character have dominated the news ever since the Iowa caucuses a few days ago. Here with us to question the candidates this evening are Josh McElveen from WMUR here in Manchester and author and comme...,intro
2,2,50,APPLAUSE,audience,752,[applause],NaN
3,3,118,MUIR,moderator,634,MUIR: So let's welcome the candidates for the Republican nomination for president. New Jersey Governor Chris Christie.,NaN
4,4,50,APPLAUSE,audience,777,[applause],NaN


In [106]:
#clean it up too
D3_df['ascii'] = D3_df['text'].apply(to_ascii)

D3_df['ascii'] = D3_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D3_df['ascii'] = D3_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D3_df['ascii'] = D3_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D3_df['ascii'] = D3_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D3_df['ascii'] = D3_df['ascii'].str.lower()

In [107]:
# Now concat all 4
D1_new = D1_df[['ascii','topic']]
T_DF = df[['ascii','subject_matter']]

T_DF = T_DF.rename(columns = {'subject_matter':'topic'})
D2_new = D2_df[['ascii','topic']]
T_new1 = pd.concat([T_DF,D1_new])
print T_new1.shape
D3_new = D3_df[['ascii','topic']]
T_new2 = pd.concat([T_new1,D2_new])
print T_new1.shape
T_new = pd.concat([T_new2,D3_new])
print T_new.shape

(4642, 2)
(4642, 2)
(6660, 2)


In [108]:
# clean the topics before encoding
# first convert to lower case
T_new['topic'] = T_new['topic'].str.lower()
a = T_new['topic'].unique()
topics = list(a)
topics.sort()
topics


[nan,
 'abortion',
 'accomplishments',
 'achieving goals',
 'afghanistan',
 'agenda',
 'benghazi',
 'billclinton',
 'bipartisanism',
 'border security',
 'bordersecurity',
 'campaign finance',
 'christie',
 'clinton',
 'clinton emails',
 'clintons emails',
 'closing',
 'cnn',
 'cnndebate',
 'cnntownhall',
 'college',
 'common core',
 'conservatism',
 'corporate america',
 'death penalty',
 'deathpenalty',
 'draft',
 'drug addiction',
 'electability',
 'eminent domain',
 'environment',
 'ethics',
 'family leave',
 'familyleave',
 'flint water',
 'flintwater',
 'flintwatercrisis',
 'foreign aid',
 'foreign policy',
 'fox news or moderators',
 'gun control',
 'healthcare (including medicare)',
 'heathcare',
 'homelandsecurity',
 'hostages',
 'immigration',
 'insiders vs outsiders',
 'intro',
 'iowa caucuses',
 'iran',
 'iran deal',
 'iraq',
 'isis',
 'israel',
 'jobs and economy',
 'lgbt and abortion',
 'lgbt issues',
 'marijuana',
 'mistreated police',
 'none',
 'none of the above',
 'no

# After seeing the list above we will combine:
 1.'clinton emails','clintons emails',
 2. 'cnn','cnndebate','cnntownhall',
 3. 'death penalty','deathpenalty'
 4. 'family leave','familyleave',
 5. 'flint water','flintwater','flintwatercrisis',
 6. 'foreign policy','north korea', 'foreign aid','afghanistan', 'homelandsecurity', 'renewed airstrikes',
 7. 'healthcare (including medicare)','heathcare', 'obamacare',
 8. 'immigration', 'border security','bordersecurity',
 9. 'iran','iran deal',
 10. 'isis', 'radical islam',
 11. 'jobs and economy', 'wall street','corporate america','taxes', 'trade agreements',
 12. 'lgbt and abortion','lgbt issues',
 13. 'marijuana', 'drug addiction',
 14. 'none','none of the above',
 15. 'torture', 'hostages',
 16. 'veterans','veterans affairs',
 17. 'women and selective service',"women's issues (not abortion though)",
 18. 'zika','zikavirus'   


In [110]:
T_new.ix[T_new['topic'] == 'clintons emails','topic'] = 'clinton emails' #1
T_new.ix[T_new['topic'].isin( ['cnndebate','cnntownhall']),'topic'] = 'cnn' #2
T_new.ix[T_new['topic'] == 'deathpenalty','topic'] = 'death penalty' #3
T_new.ix[T_new['topic'] == 'familyleave','topic'] = 'family leave'  #4
T_new.ix[T_new['topic'].isin( ['flintwater','flintwatercrisis']),'topic'] = 'flint water' #5
T_new.ix[T_new['topic'].isin( ['north korea', 'foreign aid','afghanistan', 'homelandsecurity', 'renewed airstrikes']),
         'topic'] = 'foreign policy' #6
T_new.ix[T_new['topic'].isin( ['healthcare (including medicare)','heathcare']),'topic'] = 'obamacare' #7
T_new.ix[T_new['topic'].isin( ['border security','bordersecurity']),'topic'] = 'immigration' #8
T_new.ix[T_new['topic'] == 'iran deal','topic'] = 'iran' #9
T_new.ix[T_new['topic'] == 'radical islam','topic'] = 'isis'  #10
T_new.ix[T_new['topic'].isin( ['wall street','corporate america','taxes', 'trade agreements']),'topic'] = 'jobs and economy' #11
T_new.ix[T_new['topic'] == 'lgbt and abortion','topic'] = 'lgbt issues' #12
T_new.ix[T_new['topic'] == 'drug addiction','topic'] = 'marijuana' #13
T_new.ix[T_new['topic'] == 'none of the above','topic'] = 'none'  #14
T_new.ix[T_new['topic'] == 'hostages','topic'] = 'torture'
T_new.ix[T_new['topic'] == 'veterans affairs','topic'] = 'veterans'
T_new.ix[T_new['topic'] == 'women and selective service','topic'] = "women's issues (not abortion though)"
T_new.ix[T_new['topic'] == 'zika','topic'] = 'zikavirus'



In [111]:
a = T_new['topic'].unique()
topics = list(a)
topics.sort()
topics

[nan,
 'abortion',
 'accomplishments',
 'achieving goals',
 'agenda',
 'benghazi',
 'billclinton',
 'bipartisanism',
 'campaign finance',
 'christie',
 'clinton',
 'clinton emails',
 'closing',
 'cnn',
 'college',
 'common core',
 'conservatism',
 'death penalty',
 'draft',
 'electability',
 'eminent domain',
 'environment',
 'ethics',
 'family leave',
 'flint water',
 'foreign policy',
 'fox news or moderators',
 'gun control',
 'immigration',
 'insiders vs outsiders',
 'intro',
 'iowa caucuses',
 'iran',
 'iraq',
 'isis',
 'israel',
 'jobs and economy',
 'lgbt issues',
 'marijuana',
 'mistreated police',
 'none',
 'obama',
 'obamacare',
 'priorities',
 'progressivism',
 'racial issues',
 'religion',
 'sanders campaign',
 'social security',
 'socialism',
 'syria',
 'temperament',
 'torture',
 'veterans',
 'vice presidential pick',
 'win with trump',
 "women's issues (not abortion though)",
 'zikavirus']

In [112]:
# remove intro closing nan and flipflopping 

T_new =T_new[~T_new['topic'].isin(['intro','flipflopping','closing'])]
T_new.shape

(6637, 2)

In [113]:
# set all nans in topic to none
T_new = T_new[~T_new['topic'].isnull()]

In [136]:
len(list(T_new[T_new['topic'] == "racial issues"]['ascii']))

353

In [114]:
T_new['topic'].value_counts()

none                                    1328
religion                                 407
women's issues (not abortion though)     366
racial issues                            353
abortion                                 296
jobs and economy                         284
cnn                                      228
immigration                              222
fox news or moderators                   188
billclinton                              180
iran                                     139
obamacare                                137
socialism                                137
lgbt issues                              128
isis                                     117
flint water                              106
gun control                              103
israel                                    92
benghazi                                  85
syria                                     82
foreign policy                            80
veterans                                  80
obama     

In [115]:
# Sample the data so as to get equal numbers of each class.
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(T_new['ascii'],T_new['topic'], test_size=0.3, random_state=42)


In [116]:
    
print len(X_train), len(X_test) , len(y_train) , len(y_test)

3951 1694 3951 1694


In [117]:
stop_words = pd.read_table('/Users/preetijavaji/Documents/DSE/capstone/stop_words.org',names='words')
stop_words_list = list(stop_words['w'])
from sklearn.feature_extraction.text import CountVectorizer
vectorizer   =  CountVectorizer(min_df = 1,stop_words = stop_words_list, ngram_range = (1,2))
X_f_train = vectorizer.fit_transform(X_train)

In [118]:
classifiers = []
#classifiers.append( svm.SVC() )
classifiers.append( OneVsOneClassifier(LinearSVC(random_state=0)) )
classifiers.append( OneVsRestClassifier(LinearSVC(random_state=0)) )
classifiers.append( OneVsRestClassifier(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)) )

#classifier_1 = OneVsOneClassifier(svm.SVC())
#OneVsOneClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X)

cv = 10  # Number of crossvalidation folds to use.
scores = []

for classifier in classifiers:
    scores.append( cross_validation.cross_val_score(classifier, X_f_train, y_train, cv=cv) )
    
for score in scores:
    print score
    print score.mean()
    print

print 'done'

[ 0.79146919  0.77403846  0.78883495  0.81795511  0.79345088  0.80769231
  0.8203125   0.84473684  0.83597884  0.81940701]
0.8093876095

[ 0.81753555  0.80288462  0.81067961  0.83541147  0.83375315  0.83076923
  0.84635417  0.86052632  0.86243386  0.84636119]
0.834670915364

[ 0.76540284  0.76682692  0.74029126  0.77306733  0.77078086  0.78461538
  0.79166667  0.81315789  0.82539683  0.78167116]
0.781287714735

done


/Users/preetijavaji/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


In [119]:
XT_f = vectorizer.transform(X_test)
predicteds = []
for classifier in classifiers:
    classifier.fit(X_f_train, y_train)
    yy = classifier.predict(XT_f)
    print yy.shape
    predicteds.append( classifier.predict(XT_f))


(1694,)
(1694,)
(1694,)


In [120]:
# Apply classifiers to the whole (not sampled) dataset and look at the confusion matrix and classification report for each.

# Format the whole dataset.
X_all = T_new['ascii'].values
y_all = T_new['topic'].values


# Create feature vectors from the tweets. Use the same feature vector as above.
X_featurized_all = vectorizer.transform(X_all)

# Generate predictions using the classifier trained on the equally sampled data.
predicteds = []
for classifier in classifiers:
    predicteds.append( classifier.predict(X_featurized_all) )
    
for y_pred_all in predicteds:
    print accuracy_score(y_all, y_pred_all)
    #print '   Neg ', 'Neu ', 'Pos'
    print confusion_matrix(y_all, y_pred_all)
    print classification_report(y_all, y_pred_all)
    print


0.933923826395
[[254   0   0 ...,   0  20   0]
 [  0   2   0 ...,   0   0   0]
 [  0   0   3 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,   1   0   0]
 [  6   0   0 ...,   0 334   0]
 [  0   0   0 ...,   0   0  35]]
             precision    recall  f1-score   support

   abortion       0.92      0.86      0.89       296
accomplishments       1.00      0.67      0.80         3
achieving goals       1.00      0.75      0.86         4
     agenda       1.00      1.00      1.00         3
   benghazi       0.99      1.00      0.99        85
billclinton       0.98      0.99      0.99       180
bipartisanism       1.00      1.00      1.00         1
campaign finance       1.00      0.71      0.83         7
   christie       1.00      1.00      1.00         3
    clinton       1.00      0.75      0.86         4
clinton emails       0.98      0.93      0.95        43
        cnn       0.97      0.99      0.98       228
    college       1.00      0.89      0.94        19
common core       1.00  

In [122]:
# read the tweet sample into a df
Tweet_df = pd.read_table('Tweet_g3.csv',sep=',',names=['tweetid','tweet','c3','c4','c5','c6','c7','c8'])
Tweet_df

,tweetid,tweet,c3,c4,c5,c6,c7,c8
0,NaN,"""@PinkwvPink @Pudingtane @WeepingSophia @JimBowieTexas @Myla944 @tammytabby I want my President born on U.S. soil. No Cruz! Trump 2016! """,Tweet_2016_2_2,694306691177795585,1454370589000,6.943031e+17,Mon Feb 01 23:49:49 +0000 2016,0
1,6.943067e+17,"""RT @RandellMcDonal1: @PinkwvPink @Pudingtane @WeepingSophia @JimBowieTexas @Myla944 @tammytabby I want my President born on U.S. soil. No C… """,Tweet_2016_2_2,694309252178010113,1454371200018,NaN,Tue Feb 02 00:00:00 +0000 2016,0
2,NaN,"""Yes, We Frickin Will! Vote Bernie Sanders! #IowaTODAY https://t.co/FKFVzszBls """,Tweet_2016_2_2,694207794438864896,1454347010000,NaN,Mon Feb 01 17:16:50 +0000 2016,37
3,6.942078e+17,"""RT @GlennHeiser: Yes, We Frickin Will! Vote Bernie Sanders! #IowaTODAY https://t.co/FKFVzszBls """,Tweet_2016_2_2,694309252542898176,1454371200105,NaN,Tue Feb 02 00:00:00 +0000 2016,0
4,NaN,"""So let me get this straight. If Trump see's his shadow in Iowa, we get 6 more weeks of campaigning? """,Tweet_2016_2_2,694308641080512513,1454371054000,NaN,Mon Feb 01 23:57:34 +0000 2016,3
5,6.943086e+17,"""RT @peopleofhalifax: So let me get this straight. If Trump see's his shadow in Iowa, we get 6 more weeks of campaigning? """,Tweet_2016_2_2,694309252769435648,1454371200159,NaN,Tue Feb 02 00:00:00 +0000 2016,0
6,NaN,"""@ Donald Trump https://t.co/ziNRDwb8iT """,Tweet_2016_2_2,694279114937798656,1454364014000,NaN,Mon Feb 01 22:00:14 +0000 2016,1496
7,6.942791e+17,"""RT @ParksPics: @ Donald Trump https://t.co/ziNRDwb8iT """,Tweet_2016_2_2,694309252685512704,1454371200139,NaN,Tue Feb 02 00:00:00 +0000 2016,0
8,NaN,"""Trump's ""go for a little ride"" comment shows a complete lack of understanding of the threats faced by American troops fighting today's wars. """,Tweet_2016_2_2,694305183032033280,1454370229000,NaN,Mon Feb 01 23:43:49 +0000 2016,17
9,6.943052e+17,"""RT @jrsalzman: Trump's ""go for a little ride"" comment shows a complete lack of understanding of the threats faced by American troops fighti… """,Tweet_2016_2_2,694309252513566725,1454371200098,NaN,Tue Feb 02 00:00:00 +0000 2016,0


In [123]:
# copy classifier 0 as the model for g1 and test it in spark on a sample 
# Convert to ascii
Tweet_df['ascii'] = Tweet_df['tweet'].apply(to_ascii)

# Remove hashtags
#df['ascii'] = df['ascii'].str.replace(r"#([A-Za-z0-9_]+)", " ")

# Remove handles
#df['ascii'] = df['ascii'].str.replace(r"@([A-Za-z0-9_]+)", " ")

# Remove URLs
#df['ascii'] = df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", " ")
# Replace URLs with "URL"
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")
#df['ascii'] = df['ascii'].str.replace(r"(['';:@%#()\+\*\"\…\“\”])", "")
#df['ascii'] = df['ascii'].str.replace(r"([;:@%#()\+\*\"\…\“\”])", "")

# Remove eol symbols
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbolst
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
Tweet_df['ascii'] = Tweet_df['ascii'].str.lower()


Tweet_df[['tweet', 'ascii']].head(3)
#df[df['ascii'].str.contains("\n")][['text', 'ascii']].head(10)

,tweet,ascii
0,"""@PinkwvPink @Pudingtane @WeepingSophia @JimBowieTexas @Myla944 @tammytabby I want my President born on U.S. soil. No Cruz! Trump 2016! """,@pinkwvpink @pudingtane @weepingsophia @jimbowietexas @myla944 @tammytabby i want my president born on u.s. soil. no cruz! trump 2016!
1,"""RT @RandellMcDonal1: @PinkwvPink @Pudingtane @WeepingSophia @JimBowieTexas @Myla944 @tammytabby I want my President born on U.S. soil. No C… """,rt @randellmcdonal1 @pinkwvpink @pudingtane @weepingsophia @jimbowietexas @myla944 @tammytabby i want my president born on u.s. soil. no c
2,"""Yes, We Frickin Will! Vote Bernie Sanders! #IowaTODAY https://t.co/FKFVzszBls ""","yes, we frickin will! vote bernie sanders! #iowatoday url"


In [124]:
X = Tweet_df['ascii'].values

#train_vectors = vectorizer.fit_transform(train_data)
#test_vectors = vectorizer.transform(test_data)
X_featurized = vectorizer.transform(X)
        
print X_featurized.shape
print 'done'

(10000, 37179)
done


In [125]:
predicted_new = []
for classifier in classifiers:
    predicted_new.append( classifier.predict(X_featurized) )

In [126]:
Tweet_df['topic'] = predicted_new[0]

In [131]:
from collections import Counter

Keys = Counter(predicted_new[0]).keys() # equals to list(set(words))
values =  Counter(predicted_new[0]).values() # counts the elements' frequency
for k,v in zip(Keys,values):
    print k,':',v

racial issues : 376
fox news or moderators : 63
marijuana : 2
college : 26
family leave : 4
lgbt issues : 16
obamacare : 54
environment : 3
religion : 89
clinton emails : 40
cnn : 147
billclinton : 798
veterans : 31
social security : 5
benghazi : 33
israel : 2
iran : 11
immigration : 59
foreign policy : 2
iraq : 25
obama : 209
none : 6502
isis : 25
gun control : 31
jobs and economy : 678
abortion : 101
socialism : 500
flint water : 3
syria : 2
women's issues (not abortion though) : 163


In [129]:
Tweet_df[Tweet_df['topic'] == "racial issues"]['tweet']

42                                     "RT @c0mic_sans: Sadly this is considered a progressive statement by the Republican Party  https://t.co/SpB6Y7bD4n "
157                      "RT @SarahMReports: People are trickling into the second largest GOP caucus site in the state: #IACaucus https://t.co/eQ67uNB0YX "
159                                                                           "I am A Black Man &amp; I'm Voting for Donald Trump https://t.co/LglNx6OBAT "
160                                                            "RT @wileyshow: I am A Black Man &amp; I'm Voting for Donald Trump https://t.co/LglNx6OBAT "
240       "RT @JaredWyand: Im voting for Trump because 🇺🇸 is the greatest super power the world has ever known  Time to act like it  #IowaTODAY https:… "
321           "Assumed GOP caucus similar to Dems. But GOP more like party meeting where you vote at end. Dems have their process https://t.co/PpshIZao0Z "
358                                                       "@ourown

In [137]:
# copy classifier 0 as the model for g1 and test it in spark on a sample 
import pickle
# save model in a pickle file
output = open('mymodel_g3.pkl', 'wb')
pickle.dump(classifiers[0],output)
output.close()

In [138]:
vec_file = open('myvec_g3.pkl','wb')
pickle.dump(vectorizer,vec_file)
vec_file.close()